### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [106]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    recall_score,
    precision_score,
    f1_score,
)

from time import time
import warnings

warnings.filterwarnings("ignore")

In [34]:
df = pd.read_csv("./data/dataset-minimal.csv")

In [35]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [36]:
df["humor"].value_counts(normalize=True)

humor
False    0.500215
True     0.499785
Name: proportion, dtype: float64

#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [37]:
pd.DataFrame(
    {
        "model": ["Logistic", "Decision Tree", "Bayes"],
        "best_params": ["", "", ""],
        "best_score": ["", "", ""],
    }
).set_index("model")

,best_params,best_score
model,,
Logistic,,
Decision Tree,,
Bayes,,


Pre-processing

In [101]:
X = df["text"]

In [102]:
Xt = X.apply(nltk.word_tokenize)

In [89]:
stemmer = PorterStemmer()
Xs = Xt.apply(lambda x: [stemmer.stem(xk) for xk in x]).apply(lambda x: " ".join(x))

In [107]:
lemma = WordNetLemmatizer()
Xl = Xt.apply(lambda x: [lemma.lemmatize(xk) for xk in x]).apply(lambda x: " ".join(x))

In [56]:
y = df["humor"]

In [94]:
X_train, X_test, y_train, y_test = train_test_split(
    Xs, y, random_state=42, test_size=0.25
)

In [92]:
def make_pipeline(vectorizer_name: str, classifier_name: str):
    return Pipeline(
        [
            ("vectorizer", eval(vectorizer_name)()),
            ("classifier", eval(classifier_name)()),
        ]
    )

In [133]:
preprocessors = ["none", "stemmer", "lemmatizer"]
vectorizers = ["CountVectorizer", "TfidfVectorizer"]
classifiers = ["LogisticRegression", "DecisionTreeClassifier", "MultinomialNB"]

In [122]:
all_param_grids = {
    "CountVectorizer": {
        "vectorizer__stop_words": [None, "english"],
    },
    "TfidfVectorizer": {
        "vectorizer__stop_words": [None, "english"],
    },
}

In [135]:
results_list = []

for preprocessor in preprocessors:
    if preprocessor == "none":
        Xp = X
    elif preprocessor == "stemmer":
        Xp = Xs
    elif preprocessor == "lemmatizer":
        Xp = Xl
    else:
        raise ValueError(f"Invalid choice '{preprocessor}' for preprocessor")

    X_train, X_test, y_train, y_test = train_test_split(
        Xp, y, random_state=42, test_size=0.25
    )

    for vectorizer in vectorizers:
        for classifier in classifiers:
            print(
                f"Running preprocessor = {preprocessor}, vectorizer = {vectorizer}, and classifier = {classifier}"
            )
            tic = time()
            grid = GridSearchCV(
                estimator=make_pipeline(
                    vectorizer_name=vectorizer, classifier_name=classifier
                ),
                param_grid=all_param_grids[vectorizer],
            ).fit(X_train, y_train)
            grid_search_time = time() - tic
            results_list.append(
                {
                    "preprocessor": preprocessor,
                    "vectorizer": vectorizer,
                    "classifier": classifier,
                    "average fit time": grid_search_time
                    / len(grid.cv_results_["params"]),
                    "best score": grid.best_estimator_.score(X_test, y_test),
                    "best params": grid.best_params_,
                }
            )

Running preprocessor = none, vectorizer = CountVectorizer, and classifier = LogisticRegression


In [127]:
pd.DataFrame(results_list)

,preprocessor,vectorizer,classifier,average fit time,best score,best params
0,none,CountVectorizer,LogisticRegression,11.657103,0.92400,{'vectorizer__stop_words': None}
1,none,CountVectorizer,DecisionTreeClassifier,79.656882,0.86192,{'vectorizer__stop_words': None}
2,none,CountVectorizer,MultinomialNB,3.973476,0.91276,{'vectorizer__stop_words': None}
3,none,TfidfVectorizer,LogisticRegression,7.280980,0.91676,{'vectorizer__stop_words': None}
4,none,TfidfVectorizer,DecisionTreeClassifier,133.102442,0.85016,{'vectorizer__stop_words': None}
5,none,TfidfVectorizer,MultinomialNB,4.020751,0.90980,{'vectorizer__stop_words': None}
